The following code will take in words from a file and output to a new file only the hebID and all the related definitions that are not strong's or kjv.

In [39]:
#importing the important libraries useful for the following work that will be done here.
import json
import pprint
import unicodedata
import re
from statistics import median

We will have to work with string splicing and other more inefficient methods because the json is not well formatted and I don't want to spend more time figuring out a more clever, complicated method as opposed to just going ahead with the straightforward string one.

The first part will take the exterior "Hebrew id" and put it inside the primary curly brackets and will also add a new data attribute that is "non-nikud word" (i.e. will not include Hebrew vowels)

See the following link for information regarding unicode normalization:
https://unicode.org/reports/tr15/|

In [40]:
#Create the files to that will be read and written to.
fileIn = open("definitions/unformatted_strongs.json", "r", encoding="utf-8")
fileOut = open("definitions/added_non_nikud_defs.json", "w", encoding="utf-8")
for line in fileIn:
    hebID = line[:line.index(':')]
    nikud_word = line[line.index('"lemma"') + 9 : line.index('","xlit"')]
    non_nikud_word = "".join([c for c in unicodedata.normalize('NFKD', nikud_word) if not unicodedata.combining(c)])
    new_def = f'{{ "hebID" : {hebID}, "no_nikud" : "{non_nikud_word}", '
    new_def += line[line.index('"lemma"'):len(line) - 2] + "\n"
    fileOut.write(new_def)
#Closing all the files
fileIn.close()
fileOut.close()

We want to include the morphology of the word in the definition of the word (i.e. the structure of the word [gender, plurality, etc.]). Unfortunately, that is in a different file, so we will have to splice it into the main definition.

In [41]:
fileIn = open('definitions/xml_defs.xml', 'r', encoding="utf-8")
morph_list = []
for line in fileIn:
    if('morph=' in line):
        morph = line[line.index('morph=') + 6 : line.index(' POS=')]
        morph_list.append(morph)
fileIn.close()

While the default definitions in strongs reference are good, they can be verbose and obscure. There are better definitions in the file "xml_defs.xml", so we will take those and add them to our default definitions. We will start be adding all the definitions to a list to be used later.

In [42]:
all_defs = []
fileIn = open("definitions/xml_defs.xml", 'r', encoding="utf-8")
line = fileIn.readline()
index = 1
while line:
    #If we come across a list of definitions
    if '<list>' in line:
        defs = "["
        line = fileIn.readline()
        #While there are still more definitions to be read
        while '</list>' not in line:
            line = line.strip()
            startTagIndex = line.index('<item>')
            endTagIndex = line.index('</item>')
            #Escape the quotation marks within the line.
            line = line[startTagIndex + 6 : endTagIndex].replace('"', '\\"')
            defs += f'"{line}", '
            line = fileIn.readline()
        defs = defs[:len(defs) - 2] + "]"
        all_defs.append(defs)
        defs = []
        index += 1
    line = fileIn.readline()
fileIn.close()

We will now take the definitions we have acquired along with the morphology of the word and insert it into the definition along with all other previous definition information.|

In [43]:
#Used to insert the morphology into the definitions line.
def insertMorph(main_line, morph):
    xlit_index = main_line.index('"xlit":')
    return main_line[:xlit_index] + f' "morph":{morph}, ' + main_line[xlit_index:]

#Used to insert the additional defs into the definitions line.
def insertDefs(main_line, defs):
    defs = f', "all_defs" : {defs}}}\n'
    return main_line[:len(main_line) - 2] + defs

fileIn = open("definitions/added_non_nikud_defs.json", "r", encoding = "utf-8")
fileOut = open("definitions/added_more_defs.json", "w", encoding = "utf-8")
for i in range(8674):
    main_line = fileIn.readline()
    main_line = insertMorph(main_line,morph_list[i])
    main_line = insertDefs(main_line, all_defs[i])
    main_line = main_line.replace("nikud", "niqqud", 1)
    main_line = main_line.replace("lemma", "niqqud", 1)
    fileOut.write(main_line)
fileIn.close()
fileOut.close()

Though quite inefficient now that I look back on it, we will take all the new definitions that we have put into the file added_more_defs and now add them them to a dictionary that we will use to process the definitions into a more manageable format given that there are many sub-definitions that are labeled using an inefficient system of numbers and letters as opposed to one of depths (i.e. top definition then sub-definition then another sub-definition and so on).

In [44]:
dictionary_defs = []
fileIn = open("definitions/added_more_defs.json", 'r', encoding = 'utf-8')
for word in fileIn:
    all_defs_index = word.index('"all_defs"')
    dictionary_defs.append(json.loads("{" + word[all_defs_index:]))
fileIn.close()

The following then takes in all the hebrew words and dictionary definitions and imports them into a dictionary that will process them.

The following code does several things in order to clean up all the extra definitions that we will be adding to our words.
The first thing that the code will do is that for each of the word's definitions, the definitions will remove the obsolete and single depth ordering method in favor of one that utilizes JSON objects and lists to differentiate the definitions and the sub definitions.

The first part is not entirely perfect, and leaves many empty definitions in addition to the proper ones, we will utilize a flattening algorithm that will take the definitions and remove all the empty definitions and leave only the original definitions. We are utilizing this method instead of trying to sort out the problem with the main organization algorithm because we will only run this once and the organization algorithm is not corrupting the original definitions which is the most important part.

The third part will remove the inefficient numbering still remaining and the fourth part will convert the format of definitions being contained in lists into dictionaries in order to have them be easily stored as json objects.

In [45]:
#Organize the defs of each word according to their depth
#relative to a parent/ancestor.
def organize_defs(defs):
    results = []
    #if a parentheses could not be found, make the
    #base depth 1 (i.e. Abi = "my father" (H21))
    try:
        cur_depth = len(defs[0][:defs[0].index(")")])
    except:
        cur_depth = 1
    index = 0
    #for each definition in the list
    while(index != len(defs)):
        #if a parenthesis could not be found in the def
        #simply append it to the results list and continue.
        try:
            def_depth = len(defs[index][:defs[index].index(")")])
        except:
            results.append(defs[index])
            index += 1
            continue
        #Prevents words with non-essential parentheses from having a
        #greater depth than they should. 5 was chosen because this type
        #of thing occured more when there was 5 or more depths to a word's defintions.
        if(def_depth > 5 and '=' in defs[index]):
            results.append(defs[index])
            index += 1
            continue
        #if on the same level, simply append.
        if(def_depth == cur_depth):
            results.append(defs[index])
            index += 1
        elif(def_depth > cur_depth):
            #if the depth has increased, call the function again on
            #every def after that one, inclusive. Append the resulting list.
            sub_list = organize_defs(defs[index:])
            results.append(sub_list[1])
            #if the functions have reached the end of all defs.
            if(sub_list[0] == -1):
                return (-1, results)
            #move up to where the recursive call left off.
            index += sub_list[0]
        else:
            #we return the index of where the recursive call ended
            #and the sublist to be appended.
            return (index, results)
    #return this if reached the end of the defs.
    return (-1, results)

#flattens the defs list to not include numbers
def flatten(defs):
    if(type(defs[0]) is str):
        return defs
    if(type(defs[0]) is int):
        return flatten(defs[1])
    return defs

#Removes the numbering from the definitions
def removeNumbering(defs):
    index = 0
    while(index != len(defs)):
        if(type(defs[index]) == list):
            defs[index] = removeNumbering(defs[index])
        else:
            try:
                p_index = defs[index].index(")")
            except:
                index += 1
                continue
            defs[index] = defs[index][p_index + 1:].strip()
        index += 1
    return defs

#Transform the the nesting of the definitions from lists into dictionaries.
def changeNestedTypes(defs):
    results = []
    index = 0
    for definition in defs:
        if(type(definition) is str):
            results.append({"definition" : definition})
        elif(type(definition) is list):
            sub_results = changeNestedTypes(definition)
            #print(index, sub_results)
            results.append({"senses" : sub_results})
        index += 1
    return results

#Organize all the definitions, flatten and remove
#numbering from the results
for i, word in enumerate(dictionary_defs):
    organized_defs = organize_defs(word['all_defs'])
    flattened_defs = flatten(organized_defs)
    unnumbered_defs = removeNumbering(flattened_defs)
    word['all_defs'] = changeNestedTypes(unnumbered_defs)

This last part will now replace the old definitions format with the new and improved one.

In [46]:
fileIn = open("definitions/added_more_defs.json", 'r', encoding = 'utf-8')
fileOut = open("definitions/final_defs.json", "w", encoding = 'utf-8')
for index, word in enumerate(fileIn):
    all_defs_index = word.index('"all_defs" : ')
    word_defs = json.dumps(dictionary_defs[index], ensure_ascii=False)
    fileOut.write(word[:all_defs_index] + word_defs[1:len(word_defs) - 1] + "}\n")
fileIn.close()
fileOut.close()

1/7/2021: A problem with the current definitions is that when they are in the db and a call is made to retrieve the definitions matching the word "mother", there is no ranking system to give more weight to certain words. So definitions that are a match, but whose frequency in the bible is limited can occur way before the most frequent match.

We will use a new file containing the words along with the number of references the word has to calculate the frequency of the word and improve the results returned by calls to the hebrew db.

The first task will be to organize the words since they are in hebraic-alphabetic order as opposed to being order by strong's concordance ids, which is useful for comparing our previous output in the last cell to the one here to ensure that the data isn't wonky.

In [47]:
fileIn = open('definitions/words_refs.xml', 'r', encoding='utf-8')
line = fileIn.readline()
sorted_words = []
while line:
    if "<w" in line:
        #Corresponds to strongs concordance number.
        key_value = line[line.index('a="') + 3:line.rindex("\"")]
        key_value = int(re.sub("[^0-9]", "", key_value))
        complete_line = ""
        while "</w>" not in line:
            complete_line += line
            line = fileIn.readline()
        sorted_words.append((key_value, complete_line))
    line = fileIn.readline()
sorted_words = sorted(sorted_words)
fileIn.close()

A problem with the words that we have organized is that the same word may be registered as many different words with many references, when we just need them organized by their primary form like is done in strong's concordance and thus in the previous output's we have done. We will thus consolidate the words into their main forms to make it easy to transfer the information into the document we have previously completed.

In [48]:
aggregated_words = {}
for word in sorted_words:
    if word[0] in aggregated_words:
        cur_refs = aggregated_words[word[0]]
        cur_refs += word[1]
        aggregated_words[word[0]] = cur_refs
    else:
        aggregated_words[word[0]] = word[1]
#Inefficiency exists here in that we repeat a step two times. Probably can be improved, but computes fast, so will stay.
for word in sorted(aggregated_words.items()):
    aggregated_words[word[0]] = word[1].count('<r')
aggregated_words = sorted(aggregated_words.items())

We will now take the words and insert the frequency into the main document.

In [49]:
fileIn = open('definitions/final_defs.json', 'r', encoding='utf-8')
fileOut = open('definitions/final_defs_v2.json', 'w', encoding='utf-8')
#The median frequency of all the words
median_frequency = int(median([num[1] for num in aggregated_words]))
for index, line in enumerate(fileIn):
    #Some words have no frequency apparently. If that happens, use the median frequency instead to give them some weight.
    try:
        line = line[:len(line) - 2] + f', "frequency" : {aggregated_words[index + 1][1]}}}\n'
    except IndexError:
        line = line[:len(line) - 2] + f', "frequency" : {median_frequency} }}\n'
    fileOut.write(line)
fileIn.close()
fileOut.close()